# PhotonLibrary + MLP
**This notebook: [see on github](https://github.com/drinkingkazu/2019-06-17-Notebooks/blob/master/MLP%20for%20PhotonLibrary.ipynb) or [run it on google colab](https://colab.research.google.com/github/drinkingkazu/2019-06-17-Notebooks/blob/master/MLP%20for%20PhotonLibrary.ipynb)**.

A photon library is a look-up map for a LArTPC detector. Provided (x,y,z) position, a photon library provides a probability of detecting a photon by individual PMT. Why do we use a photon library? Because simulation of individual photon propagation requires ray tracing, and it is hard to use a parametric function for approximation due to stochastic physics processes (scattering) and often a complicated detector geometry (field cage, anode wires, cryostat, etc.). A downside of using a photon library is the memory as it needs to hold infomration of probability per PMT (floating point value * PMT counts) for every single voxelized volume in the detector.

An alternative method to a photon library is a neural network, which is good at function approximation. This notebook is an attempt to emulate a photon library with MLP.

In [1]:
from __future__ import print_function
import matplotlib
%matplotlib inline
import torch
import numpy as np
SEED=123
_=np.random.seed(SEED)
_=torch.manual_seed(SEED)

! [ -d kworkshop ] || git clone -b 2019-06-17-NeuralNets https://github.com/drinkingkazu/kworkshop
! cd kworkshop && git pull && cd -

Already up to date.
/scratch/kterao/ml_at_slac/2019-06-17-NeuralNets/2019-06-17-Notebooks


Instantiate a photon library

In [2]:
from kworkshop.utils import PhotonLibrary
plib=PhotonLibrary()